In [1]:
# Celda 1: Importar librerías
import pandas as pd
import os
from tqdm import tqdm
import chardet
import numpy as np
from IPython.display import display, Markdown  # Para mejor visualización

In [2]:
# Celda 2: Función de detección de codificación
def detectar_codificacion(ruta_archivo, muestra_bytes=10000):
    """Detecta la codificación del archivo de manera más confiable usando chardet."""
    with open(ruta_archivo, 'rb') as f:
        resultado = chardet.detect(f.read(muestra_bytes))
    return resultado['encoding']

In [3]:
# Celda 3: Función para inferir tipos Oracle
def inferir_tipo_oracle(serie):
    """Infiere el tipo de dato Oracle para una columna de pandas."""
    # Convertir a string si hay valores no string
    if serie.dtype == object:
        serie = serie.astype(str)
    
    # Intentar convertir a datetime
    try:
        datetime_serie = pd.to_datetime(serie, errors='coerce')
        if not datetime_serie.isna().all():
            if any(datetime_serie.dropna().apply(lambda x: x.time() != pd.Timestamp('00:00:00').time())):
                return "TIMESTAMP"
            else:
                return "DATE"
    except:
        pass
    
    # Intentar convertir a numérico
    try:
        numeric_serie = pd.to_numeric(serie, errors='coerce')
        if not numeric_serie.isna().all():
            if (numeric_serie.dropna() % 1 == 0).all():
                max_val = numeric_serie.max()
                min_val = numeric_serie.min()
                if max_val < 10**10 and min_val > -10**10:
                    return "NUMBER(10)"
                else:
                    return "NUMBER(20)"
            else:
                return "NUMBER(15,2)"
    except:
        pass
    
    max_len = serie.str.len().max()
    return f"VARCHAR2({int(max_len)})"

In [4]:
# Celda 4: Función de análisis principal
def analizar_archivo_pandas(ruta_archivo):
    """Analiza completamente el archivo usando pandas."""
    display(Markdown("### Iniciando análisis con pandas..."))
    
    # Detectar codificación
    encoding = detectar_codificacion(ruta_archivo)
    display(Markdown(f"**Codificación detectada:** `{encoding}`"))
    
    # Determinar si el archivo es grande
    file_size = os.path.getsize(ruta_archivo)
    es_grande = file_size > 100 * 1024 * 1024  # > 100MB
    
    if es_grande:
        display(Markdown("**Archivo grande detectado.** Usando muestreo y lectura por chunks..."))
        
        # Leer encabezados
        reader = pd.read_csv(ruta_archivo, sep='|', quotechar='"', 
                           encoding=encoding, nrows=0)
        columnas = reader.columns.tolist()
        
        tipos_finales = {col: set() for col in columnas}
        max_longitudes = {col: 0 for col in columnas}
        total_filas = 0
        
        # Leer por chunks con barra de progreso
        chunksize = 10**5
        for chunk in tqdm(pd.read_csv(ruta_archivo, sep='|', quotechar='"', 
                                     encoding=encoding, chunksize=chunksize,
                                     low_memory=False, on_bad_lines='warn'),
                          desc="Procesando chunks"):
            total_filas += len(chunk)
            
            for col in columnas:
                col_limpia = col.strip().strip('"')
                serie = chunk[col].astype(str)
                current_max = serie.str.len().max()
                if current_max > max_longitudes[col_limpia]:
                    max_longitudes[col_limpia] = current_max
                tipo = inferir_tipo_oracle(serie)
                tipos_finales[col_limpia].add(tipo)
        
        display(Markdown(f"\n**Total de filas procesadas:** `{total_filas:,}`"))
    else:
        display(Markdown("**Archivo pequeño detectado.** Leyendo todo en memoria..."))
        df = pd.read_csv(ruta_archivo, sep='|', quotechar='"', 
                         encoding=encoding, low_memory=False, 
                         on_bad_lines='warn')
        
        columnas = df.columns.tolist()
        tipos_finales = {}
        max_longitudes = {}
        
        for col in columnas:
            col_limpia = col.strip().strip('"')
            serie = df[col].astype(str)
            max_longitudes[col_limpia] = serie.str.len().max()
            tipos_finales[col_limpia] = {inferir_tipo_oracle(serie)}
    
    # Consolidar tipos
    tipos_consolidados = {}
    for col, tipos in tipos_finales.items():
        if len(tipos) == 1:
            tipos_consolidados[col] = tipos.pop()
        else:
            if 'TIMESTAMP' in tipos:
                tipos_consolidados[col] = 'TIMESTAMP'
            elif 'DATE' in tipos:
                tipos_consolidados[col] = 'DATE'
            elif any(t.startswith('NUMBER') for t in tipos):
                has_decimal = any('NUMBER(15,2)' in t for t in tipos)
                tipos_consolidados[col] = 'NUMBER(15,2)' if has_decimal else 'NUMBER(10)'
            else:
                max_len = max(int(t.split('(')[1].split(')')[0]) for t in tipos if t.startswith('VARCHAR'))
                tipos_consolidados[col] = f'VARCHAR2({max_len})'
    
    return max_longitudes, tipos_consolidados

In [5]:
# Celda 5: Función para generar estructura Oracle
def generar_estructura_oracle(nombre_tabla, max_longitudes, tipos_dato):
    """Genera la estructura Oracle optimizada con formato Markdown."""
    estructura = [f"# ESTRUCTURA ORACLE GENERADA\n```sql\nCREATE TABLE {nombre_tabla.upper()} ("]
    
    for campo in sorted(max_longitudes.keys()):
        tipo = tipos_dato[campo]
        nombre_col = campo[:30].upper()
        estructura.append(f"    {nombre_col.ljust(30)} {tipo},")
    
    estructura[-1] = estructura[-1].rstrip(',')
    estructura.append(");\n```")
    
    return "\n".join(estructura)

In [ ]:
# Celda 6: Interfaz de usuario con widgets (opcional pero recomendado)
from ipywidgets import interact, widgets

def interfaz_analizador():
    display(Markdown("# Analizador de Archivos para Oracle"))
    
    # Widgets de entrada
    ruta = widgets.Text(
        value='',
        placeholder='Introduce la ruta del archivo',
        description='Ruta:',
        disabled=False
    )
    
    nombre_tabla = widgets.Text(
        value='usuarios',
        placeholder='Nombre de tabla',
        description='Tabla:',
        disabled=False
    )
    
    display(ruta, nombre_tabla)
    
    def on_button_click(b):
        display(Markdown("## Resultados del Análisis"))
        try:
            max_len, tipos = analizar_archivo_pandas(ruta.value)
            
            if max_len and tipos:
                display(Markdown("### Resumen de Columnas"))
                resumen = pd.DataFrame({
                    'Columna': list(max_len.keys()),
                    'Long. Máx': list(max_len.values()),
                    'Tipo Oracle': list(tipos.values())
                })
                display(resumen)
                
                estructura = generar_estructura_oracle(nombre_tabla.value, max_len, tipos)
                display(Markdown("### Estructura SQL Generada"))
                display(Markdown(estructura))
        except Exception as e:
            display(Markdown(f"**Error:** `{str(e)}`"))
    
    button = widgets.Button(description="Analizar Archivo")
    button.on_click(on_button_click)
    display(button)

# Ejecutar la interfaz
interfaz_analizador()

# Analizador de Archivos para Oracle

Text(value='', description='Ruta:', placeholder='Introduce la ruta del archivo')

Text(value='usuarios', description='Tabla:', placeholder='Nombre de tabla')

Button(description='Analizar Archivo', style=ButtonStyle())

## Resultados del Análisis

### Iniciando análisis con pandas...

**Codificación detectada:** `ISO-8859-1`

**Archivo grande detectado.** Usando muestreo y lectura por chunks...